In [11]:
import numpy as np
import pandas as pd
import string
np.random.seed(1234)


In [78]:
initial = {} # start of a phrase
first_order = {} # second word only
second_order = {}
def remove_punctuation(s):
    return s.translate(str.maketrans('','',string.punctuation))

In [79]:
def add2dict(d, k, v):
  if k not in d:
    d[k] = []
  d[k].append(v)


In [80]:
input_files = [
  'edgar_allan_poe.txt',
  'robert_frost.txt',
]

In [81]:
for line in open('robert_frost.txt'):
  tokens = remove_punctuation(line.rstrip().lower()).split()
  # print(" these are the tokens of ->",tokens)
  T = len(tokens)
  
  for i in range(T):
    t = tokens[i]
    if i == 0:
      # measure the distribution of the first word
      initial[t] = initial.get(t, 0.) + 1
    else:
      t_1 = tokens[i-1]
      if i == T - 1:
        # measure probability of ending the line show that loop will not go
        add2dict(second_order, (t_1, t), 'END')
      if i == 1:
        # measure distribution of second word
        # given only first word
        add2dict(first_order, t_1, t)
      else:
        t_2 = tokens[i-2]
        add2dict(second_order, (t_2, t_1), t)

print("lenght of token for the first line",T)

lenght of token for the first line 10


In [82]:
# normalize the distributions
initial_total = sum(initial.values())
for t, c in initial.items():
    initial[t] = c / initial_total

In [83]:
# convert [cat, cat, cat, dog, dog, dog, dog, mouse, ...]
# into {cat: 0.5, dog: 0.4, mouse: 0.1}

def list2pdict(ts):
  # turn each list of possibilities into a dictionary of probabilities
  d = {}
  n = len(ts)
  for t in ts:
    d[t] = d.get(t, 0.) + 1
  for t, c in d.items():
    d[t] = c / n
  return d

In [84]:
for t_1, ts in first_order.items():
  # replace list with dictionary of probabilities
  first_order[t_1] = list2pdict(ts)

In [85]:
for k, ts in second_order.items():
  second_order[k] = list2pdict(ts)

In [86]:
def sample_word(d):
  # print "d:", d
  p0 = np.random.random()
  # print "p0:", p0
  cumulative = 0
  for t, p in d.items():
    cumulative += p
    if p0 < cumulative:
      return t
  assert(False) # should never get here

In [87]:
def generate():
  for i in range(4): # generate 4 lines
    sentence = []

    # initial word
    w0 = sample_word(initial)
    sentence.append(w0)

    # sample second word
    w1 = sample_word(first_order[w0])
    sentence.append(w1)

    # second-order transitions until END
    while True:
      w2 = sample_word(second_order[(w0, w1)])
      if w2 == 'END':
        break
      sentence.append(w2)
      w0 = w1
      w1 = w2
    print(' '.join(sentence))

In [89]:
generate()

smoothlaid like thatch with the night the bones i knew in heaven
her hardest hue to hold
the night for shelter at a house
describing rings of lantern light
